In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://www.bfv.de/wettbewerbe/meisterschaften/kreisklasse-5-bt-ku/02M4RLIUU0000006VS5489B4VUAB0UC4-G'
page = requests.get(url) #200 is fine
soup = BeautifulSoup(page.text, 'html')
page

<Response [200]>

In [3]:
table = soup.find_all('table')[0]
titles = table.find_all('th')
titles = [t.text.strip() for t in titles] #list comprehension
titles

['Pl.', 'Verein', 'Sp.', 'G', 'U', 'V', 'Torv.', 'Tordiff.', 'Pkt.']

In [4]:
# read in data
import pandas as pd
df = pd.DataFrame(columns = ['Platz', 'Verein', 'Spiele', 'S', 'U', 'N', 'Torverhältnis', 'Tordifferenz', 'Punkte']) 
col_data = table.find_all('tr')
for row in col_data[1:]:
        row_data = row.find_all('td')
        ind_row_data = [r.text.strip() for r in row_data]
        
        #insert in dataframe
        length = len(df)
        df.loc[length] = ind_row_data
        
df['G'] = df['Torverhältnis'].str.split(':').str[0].astype(int)
df['A'] = df['Torverhältnis'].str.split(':').str[1].astype(int)
df.insert(6, 'G', df.pop('G'))
df.insert(7, 'A', df.pop('A'))
df['Platz'] = df['Platz'].str.strip('.')
df['Spiele'] = df['Spiele'].astype(int)
df['S'] = df['S'].astype(int)
df['U'] = df['U'].astype(int)
df['N'] = df['N'].astype(int)
df['Punkte'] = df['Punkte'].astype(int)
df.set_index('Platz', inplace=True)
df

,Verein,Spiele,S,U,N,G,A,Torverhältnis,Tordifferenz,Punkte
Platz,,,,,,,,,,
1,1. FC Bayreuth,19,15,0,4,74,18,74:18,56,45
2,TSV Engelmannsreuth,19,13,1,5,47,20,47:20,27,40
3,TSV Obernsees,18,11,4,3,50,23,50:23,27,37
4,SG Trockau,17,10,5,2,44,27,44:27,17,35
5,TSV St. Johannis,17,9,4,4,42,24,42:24,18,31
6,SV Seybothenreuth,17,9,2,6,28,22,28:22,6,29
7,Eintracht Bayreuth,16,8,3,5,27,20,27:20,7,27
8,SV Schreez,18,7,3,8,38,36,38:36,2,24
9,(SG 1) TSV Bindlach I / SV Ramsenthal I,18,6,6,6,33,36,33:36,-3,24


In [5]:
df2 = df.copy()
df2['Tore pro Spiel'] = ((df2['G'] + df2['A']) / df2['Spiele']).round(2)
df2[['Verein', 'Spiele', 'Torverhältnis', 'Tore pro Spiel']]
df2.sort_values('Tore pro Spiel', ascending=False)

,Verein,Spiele,S,U,N,G,A,Torverhältnis,Tordifferenz,Punkte,Tore pro Spiel
Platz,,,,,,,,,,,
1,1. FC Bayreuth,19,15,0,4,74,18,74:18,56,45,4.84
15,TSV Kirchenlaibach-Speichersd. II,18,3,2,13,16,63,16:63,-47,11,4.39
16,(SG 1) ASV Nemmersdorf I/SpVgg Goldkronach II,18,0,3,15,19,59,19:59,-40,3,4.33
11,(SG 1) SC Hummeltal I/TSV Glashütten I,19,7,1,11,32,49,32:49,-17,22,4.26
4,SG Trockau,17,10,5,2,44,27,44:27,17,35,4.18
8,SV Schreez,18,7,3,8,38,36,38:36,2,24,4.11
3,TSV Obernsees,18,11,4,3,50,23,50:23,27,37,4.06
5,TSV St. Johannis,17,9,4,4,42,24,42:24,18,31,3.88
10,SV Mistelgau II,19,6,4,9,32,41,32:41,-9,22,3.84


In [6]:
url = 'https://www.bfv.de/wettbewerbe/meisterschaften/kreisklasse-5-bt-ku/02M4RLIUU0000006VS5489B4VUAB0UC4-G#spieltag=1'
page = requests.get(url) #200 is fine
soup = BeautifulSoup(page.text, 'html')
page

<Response [200]>

In [7]:
x = soup.find_all('div', class_='bfv-matchdata-result__team-name bfv-matchdata-result__team-name--team0')
x = [i.text.strip() for i in x]

y = soup.find_all('div', class_='bfv-matchdata-result__team-name bfv-matchdata-result__team-name--team1')
y = [i.text.strip() for i in y]

print(x[11], y[11])

TSV Obernsees SSV Kirchenpingarten


In [8]:
table = soup.find('div', class_='bfv-spieltag-matchups__match-container')
#titles = table.find_all('th')
#titles = [t.text.strip() for t in titles] #list comprehension
#titles
game = table.find('div', class_='bfv-spieltag-eintrag')
time = game.find('div', class_='bfv-spieltag-eintrag__date-info')
day = time.find_all('span')[0].text.strip()
date = time.find_all('span')[1].text.strip()
hometeam = game.find('div', class_='bfv-matchdata-result__team-name bfv-matchdata-result__team-name--team0').text.strip()
awayteam = game.find('div', class_='bfv-matchdata-result__team-name bfv-matchdata-result__team-name--team1').text.strip()
print(hometeam, awayteam)

Universitäts-Sportclub Bayreuth TSV Engelmannsreuth


In [9]:
import pandas as pd
df = pd.DataFrame(columns = ['Platz', 'Verein', 'Spiele']) 
col_data = table.find_all('tr')
for row in col_data[1:]:
        row_data = row.find_all('td')
        ind_row_data = [r.text.strip() for r in row_data]
        
        #insert in dataframe
        length = len(df)
        df.loc[length] = ind_row_data

In [10]:
df

,Platz,Verein,Spiele


In [11]:
url = 'https://www.bfv.de/wettbewerbe/meisterschaften/kreisklasse-5-bt-ku/02M4RLIUU0000006VS5489B4VUAB0UC4-G#tabelle'
page = requests.get(url) #200 is fine
soup = BeautifulSoup(page.text, 'html')
page

<Response [200]>

In [14]:
table = soup.find_all('table')[0]
titles = table.find_all('th')
titles = [t.text.strip() for t in titles] #list comprehension
titles
# read in data
import pandas as pd
df = pd.DataFrame(columns = ['Platz', 'Verein', 'Spiele', 'S', 'U', 'N', 'Torverhältnis', 'Tordifferenz', 'Punkte']) 
col_data = table.find_all('tr')
for row in col_data[1:]:
        row_data = row.find_all('td')
        ind_row_data = [r.text.strip() for r in row_data]
        
        #insert in dataframe
        length = len(df)
        df.loc[length] = ind_row_data
        
df['G'] = df['Torverhältnis'].str.split(':').str[0].astype(int)
df['A'] = df['Torverhältnis'].str.split(':').str[1].astype(int)
df.insert(6, 'G', df.pop('G'))
df.insert(7, 'A', df.pop('A'))
df['Platz'] = df['Platz'].str.strip('.')
df['Spiele'] = df['Spiele'].astype(int)
df['S'] = df['S'].astype(int)
df['U'] = df['U'].astype(int)
df['N'] = df['N'].astype(int)
df['Punkte'] = df['Punkte'].astype(int)
df.set_index('Platz', inplace=True)
df

,Verein,Spiele,S,U,N,G,A,Torverhältnis,Tordifferenz,Punkte
Platz,,,,,,,,,,
1,1. FC Bayreuth,19,15,0,4,74,18,74:18,56,45
2,TSV Engelmannsreuth,19,13,1,5,47,20,47:20,27,40
3,TSV Obernsees,18,11,4,3,50,23,50:23,27,37
4,SG Trockau,17,10,5,2,44,27,44:27,17,35
5,TSV St. Johannis,17,9,4,4,42,24,42:24,18,31
6,SV Seybothenreuth,17,9,2,6,28,22,28:22,6,29
7,Eintracht Bayreuth,16,8,3,5,27,20,27:20,7,27
8,SV Schreez,18,7,3,8,38,36,38:36,2,24
9,(SG 1) TSV Bindlach I / SV Ramsenthal I,18,6,6,6,33,36,33:36,-3,24
